In [ ]:
import numpy as np
from datetime import datetime 
import torch
import torch.nn as nn
from torch.nn import functional as F
from torch.utils.data import DataLoader
from torchvision import datasets, transforms
import torch.quantization
from torch.utils.tensorboard import SummaryWriter
import torchvision
import matplotlib.pyplot as plt
import ipdb
import time
from lenet import LeNet5
from utils import get_accuracy, get_weights_biases, biggest_weight
# check device
DEVICE = 'cuda'

In [ ]:
# parameters
RANDOM_SEED = 42
LEARNING_RATE = 0.001
BATCH_SIZE = 128
num_workers = 10

IMG_SIZE = 32
N_CLASSES = 10

In [ ]:
transform = transforms.Compose([transforms.Resize((32, 32)), transforms.ToTensor()])
valid_dataset = datasets.MNIST(root='mnist_data', train=False,transform=transform)
valid_loader = DataLoader(dataset=valid_dataset, batch_size=BATCH_SIZE, shuffle=False, num_workers=num_workers, pin_memory=True)
test_loader = DataLoader(dataset=valid_dataset, batch_size=1, shuffle=True, num_workers=num_workers, pin_memory=True)

In [ ]:
model = LeNet5(10)#.to(DEVICE)
model.load_state_dict(torch.load("quartz-lenet.pth", map_location=torch.device('cpu')))
#model.eval()

In [ ]:
get_accuracy(model, valid_loader, device='cpu')

In [ ]:
weights, biases = get_weights_biases(model)
max_weight = np.max([weight.max() for weight in weights])
min_weight = np.min([weight.min() for weight in weights])

In [ ]:
weights[0].max()

In [ ]:
weights = [weight*2 for weight in weights]

In [ ]:
model.qconfig = torch.quantization.default_qconfig
print(model.qconfig)
torch.quantization.prepare(model, inplace=True)

In [ ]:
torch.quantization.convert(model, inplace=True)

In [ ]:
model.conv1.weight = nn.Parameter(weights[0])
model.conv2.weight = nn.Parameter(weights[1])
model.conv3.weight = nn.Parameter(weights[2])
model.fc1.weight = nn.Parameter(weights[3])
model.fc2.weight = nn.Parameter(weights[4])

In [ ]:
for param_tensor in model.state_dict():
    print(param_tensor, "\t", model.state_dict()[param_tensor].size())

In [ ]:
model.conv1.weight

In [ ]:
255 * 0.0808